In [ ]:
!unzip "/content/drive/MyDrive/fine_tuned_led_large_16384.zip" -d "/content/drive/MyDrive/"

Archive:  /content/drive/MyDrive/fine_tuned_led_large_16384.zip
  inflating: /content/drive/MyDrive/fine_tuned_led_large_16384/config.json  
  inflating: /content/drive/MyDrive/fine_tuned_led_large_16384/generation_config.json  
  inflating: /content/drive/MyDrive/fine_tuned_led_large_16384/model.safetensors  


In [ ]:
!pip install transformers[torch]

In [ ]:
##############   Preprocessing    #####################
# i have run this code in jupiter .. don't run here
#output file is saved in drive
#2)

import os
import csv
import re
from nltk.tokenize import word_tokenize

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9\s$€£¥\n\r]", "", text)
    text = re.sub(r"\s+", " ", text)
    return text

def extract_tokens(text, max_tokens):
    tokens = word_tokenize(text)
    return tokens[:max_tokens]

def divide_text_into_parts(tokens, num_parts):
    part_size = len(tokens) // num_parts
    divided_parts = [tokens[i * part_size: (i + 1) * part_size] for i in range(num_parts)]
    return divided_parts

def find_best_summary(report_tokens, golden_summaries):
    best_summary = None
    best_intersection = 0
    selected_summary_number = None

    for summary_index, summary in enumerate(golden_summaries, start=1):
        intersection = len(set(report_tokens) & set(summary))
        if intersection > best_intersection:
            best_intersection = intersection
            best_summary = summary
            selected_summary_number = summary_index

    return best_summary, selected_summary_number

dataset_folder = "fns_2020/validation"
output_csv_file = "val_preprocessed_data.csv"

count = 0
error_files = []

with open(output_csv_file, mode='w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['input_report', 'selected_summary_number', 'generated_summary', 'report_file_path'])

    for report_file in os.listdir(os.path.join(dataset_folder, 'annual_reports')):

        report_path = os.path.join(dataset_folder, 'annual_reports', report_file)
        try:
            with open(report_path, 'r', encoding='utf-8') as f:
                report_text = f.read()
                report_text = preprocess_text(report_text)
                report_tokens = extract_tokens(report_text, 8192)

            golden_summary_folder = os.path.join(dataset_folder, 'gold_summaries')
            golden_summaries = []
            for summary_file in os.listdir(golden_summary_folder):
                if summary_file.startswith(os.path.splitext(report_file)[0]):
                    summary_path = os.path.join(golden_summary_folder, summary_file)
                    with open(summary_path, 'r', encoding='utf-8') as f:
                        summary_text = f.read()
                        summary_text = preprocess_text(summary_text)
                        summary_tokens = extract_tokens(summary_text, 1024)
                        golden_summaries.append(summary_tokens)

            best_summary_tokens, selected_summary_number = find_best_summary(report_tokens, golden_summaries)
            divided_report_parts = divide_text_into_parts(report_tokens, 8)

            for i, report_part in enumerate(divided_report_parts, start=1):
                csv_writer.writerow([' '.join(report_part), selected_summary_number, ' '.join(best_summary_tokens), report_file])
                count += 1
                print(count)

        except Exception as e:
            print(f"Error processing file {report_path}: {str(e)}")
            error_files.append(report_path)

In [ ]:
import pandas as pd
from transformers import LEDForConditionalGeneration, LEDTokenizer


In [ ]:


#from datasets import Dataset, load_dataset
from torch.utils.data import DataLoader
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load input data and generated summaries from CSV
data_df = pd.read_csv('/content/drive/MyDrive/preprocessed_data.csv')  # Update with the path to your CSV file

#-----------------------------------------------------------------------------------------------------------------------


In [ ]:
# Assuming your CSV file has columns named 'input_data' and 'generated_summary'
input_data = data_df['input_report'].tolist()
generated_summary = data_df['generated_summary'].tolist()
print(f"Input data {input_data}")
# Tokenization
tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")


# Define the maximum input and output lengths
max_input_length = 1024
max_output_length = 128
# Tokenize input and output data
#tokenized_data = tokenizer(input_data, generated_summary, max_length=max_input_length, padding="max_length", truncation=True)
# Tokenize input and output data
tokenized_data = tokenizer(input_data, max_length=max_input_length, padding="max_length", truncation=True, return_tensors="pt")

# Update tokenized_data to include labels
tokenized_data['labels'] = tokenizer(generated_summary, max_length=max_output_length, padding="max_length", truncation=True, return_tensors="pt")['input_ids']

# Ensure that the tokenized data contains 'input_ids' and 'labels'
input_ids = tokenized_data['input_ids']
labels = tokenized_data['labels']

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Load input data and generated summaries from CSV
val_data_df = pd.read_csv('/content/drive/MyDrive/val_preprocessed_data.csv')  # Update with the path to your CSV file

In [ ]:

# Assuming your CSV file has columns named 'input_data' and 'generated_summary'
val_input_data = val_data_df['input_report'].tolist()
val_generated_summary = val_data_df['generated_summary'].tolist()
print(f"Input data {val_input_data}")
# # Tokenization
# tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")

# Define the maximum input and output lengths
max_input_length = 1024
max_output_length = 128
# Tokenize input and output data
#tokenized_data = tokenizer(input_data, generated_summary, max_length=max_input_length, padding="max_length", truncation=True)
# Tokenize input and output data
val_tokenized_data = tokenizer(val_input_data, max_length=max_input_length, padding="max_length", truncation=True, return_tensors="pt")

# Update tokenized_data to include labels
val_tokenized_data['labels'] = tokenizer(val_generated_summary, max_length=max_output_length, padding="max_length", truncation=True, return_tensors="pt")['input_ids']


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# don't run this command if u have already installed it
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from datasets import Dataset

# Prepare the dataset
dataset = Dataset.from_dict(tokenized_data)
val_dataset = Dataset.from_dict(val_tokenized_data)

In [ ]:
# # Model Configuration
# model = LEDForConditionalGeneration.from_pretrained("allenai/led-base-16384")

model_name_or_path = "/content/drive/MyDrive/fine_tuned_led_large_16384"

# Load tokenizer
tokenizer = LEDTokenizer.from_pretrained("allenai/led-base-16384")


# Load model
model = LEDForConditionalGeneration.from_pretrained(model_name_or_path)


# Fine-Tuning
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=500,  # Evaluate every 500 steps
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=val_dataset,  # Specify the evaluation dataset
)

In [ ]:
#training
trainer.train()

# Save the Model
model.save_pretrained("fine_tuned_led_large_16384")


In [ ]:
pd=trainer.predict(val_dataset)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
print(pd)

PredictionOutput(predictions=array([[    2,     0,  3387, ...,   266,    52,   548],
       [    2,     0,  4124, ...,  6149,    29,  3744],
       [    2,     0, 15922, ...,   266,     8,  2349],
       ...,
       [    2,     0,  6031, ...,   834,  4585,  1551],
       [    2,     0,  1570, ...,    52,    33,    10],
       [    2,     0,  1366, ...,   806,    24,    70]]), label_ids=array([[    0,   698, 14382, ...,  7482,  1633,     2],
       [    0,   698, 14382, ...,  7482,  1633,     2],
       [    0,   698, 14382, ...,  7482,  1633,     2],
       ...,
       [    0,  4124, 13550, ...,  3156,    11,     2],
       [    0,  4124, 13550, ...,  3156,    11,     2],
       [    0,  4124, 13550, ...,  3156,    11,     2]]), metrics={'test_loss': 2.970895528793335, 'test_runtime': 516.8134, 'test_samples_per_second': 5.619, 'test_steps_per_second': 0.702})


In [ ]:
######

In [ ]:
#working fine to add one cell in test.csv file

# Load "val_preprocessed_data.csv" and get the "generated_summary" and "report_file_path" columns
with open("/content/drive/MyDrive/val_preprocessed_data.csv", "r", encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    rows = list(csv_reader)
    generated_summary = rows[0]["generated_summary"]  # Take the first generated summary
    report_file_path = rows[0]["report_file_path"]  # Take the first report file path

# Example predictions and label_ids
# pd = trainer.predict(val_dataset)
predictions = pd.predictions
label_ids = pd.label_ids

# Decode the first 8 arrays in label_ids
decoded_text = ""
for labels in label_ids[:8]:
    decoded_text += tokenizer.decode(labels)

# Write data to the CSV file
with open("test.csv", "w", newline="", encoding='utf-8') as csvfile:
    fieldnames = ["filename", "generated_text", "generated_summary"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Write the first filename, decoded text, and generated summary to the CSV file
    writer.writerow({"filename": report_file_path, "generated_text": decoded_text, "generated_summary": generated_summary})


In [ ]:
#this code is working fine for test.csv
import csv

# Load "val_preprocessed_data.csv" and get the "generated_summary" and "report_file_path" columns
with open("/content/drive/MyDrive/val_preprocessed_data.csv", "r", encoding='utf-8') as csvfile:
    csv_reader = csv.DictReader(csvfile)
    rows = list(csv_reader)

# Example predictions and label_ids
# pd = trainer.predict(val_dataset)
predictions = pd.predictions
label_ids = pd.label_ids

# Write data to the CSV file
with open("test.csv", "w", newline="", encoding='utf-8') as csvfile:
    fieldnames = ["filename", "generated_text", "generated_summary"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    idx = 0
    while idx < len(rows):
        # Take one value from "report_file_path" and "generated_summary" from the current set of rows
        report_file_path = rows[idx]["report_file_path"]
        generated_summary = rows[idx]["generated_summary"]

        # Decode the next 8 arrays in label_ids
        decoded_text = ""
        for labels in label_ids[idx:idx+8]:
            decoded_text += tokenizer.decode(labels)

        # Write the filename, decoded text, and generated summary to the CSV file
        writer.writerow({"filename": report_file_path, "generated_text": decoded_text, "generated_summary": generated_summary})

        # Move to the next set of 8 rows in val_preprocessed_data.csv
        idx += 8


In [ ]:
#rouge scores calculation

In [ ]:
import csv
!pip install rouge-score


In [ ]:
from rouge_score import rouge_scorer

# Function to calculate all ROUGE scores
def calculate_rouge_scores(generated_text, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(generated_text, generated_summary)
    rouge_scores = f"ROUGE-1: {scores['rouge1'].fmeasure:.4f}, ROUGE-2: {scores['rouge2'].fmeasure:.4f}, ROUGE-L: {scores['rougeL'].fmeasure:.4f}"
    return rouge_scores


In [ ]:
#working fine
# Read data from the existing CSV file
input_csv_file = "/content/drive/MyDrive/test.csv"
output_csv_file = "updated_file.csv"

with open(input_csv_file, "r", encoding="utf-8") as csv_file:
    csv_reader = csv.DictReader(csv_file)
    rows = list(csv_reader)

# Update data and write to the new CSV file
with open(output_csv_file, "w", newline="", encoding="utf-8") as csv_file:
    fieldnames = ["filename", "generated_text", "generated_summary", "rouge"]
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()

    for row in rows:
        # Calculate all ROUGE scores for each row
        rouge_scores = calculate_rouge_scores(row["generated_text"], row["generated_summary"])

        # Update the row with all ROUGE scores
        row["rouge"] = rouge_scores

        # Write the updated row to the new CSV file
        csv_writer.writerow(row)

print("CSV file updated successfully with all ROUGE scores.")